In [1]:
import numpy as np
import pandas as pd
import shapefile

from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
from scipy.interpolate import splprep, splev
from sklearn.decomposition import PCA

from bokeh.plotting import figure, show
from bokeh.palettes import Viridis256, Inferno256
from bokeh.transform import linear_cmap
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column
from bokeh.io import output_notebook    # In-notebook visualisation
output_notebook()

Loading BokehJS ...

In [16]:
class Albers:
    '''A class to carry out Albers equal-area conical projections'''
    
    def __init__(self, x0, y0, y1, y2):
        '''degrees : If True, the other parameters are given in degrees; otherwise radians
           x0, y0  : Reference longitude, latitude (coordinates of origin)
           y1, y2  : Standard parallels'''
        self.x0, self.y0, self.y1, self.y2 = np.array([x0, y0, y1, y2])*np.pi/180
        self.R = 6371 # Radius of Earth in kilometres
        self.n  = (np.sin(self.y1) + np.sin(self.y2)) / 2
        self.c  = np.cos(self.y1)**2 + 2*self.n*np.sin(self.y1)
        self.p0 = (self.R/self.n)*np.sqrt(self.c - 2*self.n*np.sin(self.y0))
        return None
    
    def scale(self, array):
        return (self.R/self.n)*np.sqrt(self.c - 2*self.n*np.sin(array))
    
    def convert(self, coords):
        array = coords*(np.pi/180)
        p = self.scale(array[:,1])
        t = self.n*(array[:,0] - self.x0)
        u = p*np.sin(t)
        v = self.p0 - p*np.cos(t)
        return np.c_[u,v]
    

def perimeter(coords, closed=True):
    '''Measure perimeter of polygon'''
    side_vectors = coords - np.roll(coords, 1, axis=0)
    if closed:
        lengths = np.linalg.norm(side_vectors, axis=1)
    else:
        lengths = np.linalg.norm(side_vectors[1:], axis=1)
    return np.sum(lengths)

In [17]:
A = Albers(0, 51.5, 51, 56)

shape_path = 'GBR_adm0.shp'
shape_obj  = shapefile.Reader(shape_path)
raw_polys  = np.array(shape_obj.shapes()[0].points)
all_polys  = np.split(raw_polys, shape_obj.shapes()[0].parts)[1:]
poly_num   = len(all_polys)
poly_lengths = [len(poly) for poly in all_polys]
indices      = np.argsort(poly_lengths)
all_polys    = [all_polys[indices[poly_num-i-1]] for i in range(poly_num)]
all_polys  = [A.convert(poly) for poly in all_polys]
perimeters = [perimeter(poly) for poly in all_polys]

In [18]:
arran = all_polys[15]
p = figure(match_aspect=True)
p.line(arran[:,0], arran[:,1])
show(p)